In [1]:
from torchvision import models
from torch.autograd import Variable

## Utilities
import random

## Libraries
import numpy as np


## Torch
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn

In [2]:
## Normalization on ImageNet mean/std for finetuning
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

save_dir = './snapshots'
batch_size = 64

In [3]:
# Setting random seeds for reproducibility. (Caveat, some CuDNN algorithms are non-deterministic)
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)
np.random.seed(1337)
random.seed(1337)

In [4]:
## Normalization only for validation and test
ds_transform_raw = transforms.Compose([
                     transforms.CenterCrop(224),
                     transforms.ToTensor(),
                     normalize
                     ])

In [57]:
# Load model from best iteration
from src.p_neuro import ResNet50

class CNN(nn.Module):
    ## We use ResNet weights from PyCaffe.
    def __init__(self, embed_size):
        super(CNN, self).__init__()
        
        # Loading pretrained ResNet as feature extractor
        original_model = ResNet50(17)
        model_path = './snapshots/2017-05-06_1235-cloud-habitation-PowerPIL-model_best.pth'
        checkpoint = torch.load(model_path)
        original_model.load_state_dict(checkpoint['state_dict'])
        
        # Everything except the last linear layer
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        
        # Freeze those weights
        for p in self.features.parameters():
            p.requires_grad = False

        # Get number of features of last layer
        num_feats = original_model.classifier[0].in_features
            
        self.fc = nn.Linear(num_feats, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)

    def forward(self, x):
        f = self.features(x)
        f = f.view(f.size(0), -1)
        out = self.fc(f)
        out = self.bn(out)
        return out

In [58]:
encoderCNN = CNN(2048).cuda()

In [59]:
from torch.nn.init import kaiming_normal

In [94]:
class DecoderRNN(nn.Module):
    def __init__(self, num_feats, num_classes, hidden_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.rnn = nn.GRU(input_size=num_feats,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first = True)
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.hidden_size = hidden_size
        
        # Init of last layer
        kaiming_normal(self.classifier.weight)
    

    def forward(self, feats, hidden=None):
        x, hidden = self.rnn(feats.unsqueeze(1), hidden)
        x = x.view(-1, self.hidden_size)
        x = self.classifier(x)
        return x

In [95]:
decoderRNN = DecoderRNN(2048, 17, 64, 10).cuda()

In [96]:
## Normalization on ImageNet mean/std for finetuning
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

# Note, p_training has lr_decay automated
optimizer = optim.SGD(decoderRNN.parameters(), lr=1e-1, momentum=0.9) # Finetuning whole model

# criterion = ConvolutedLoss()
criterion = torch.nn.MultiLabelSoftMarginLoss(
    weight = torch.from_numpy(
                 1/np.array([1,  3,  2,  1,
                             1,  3,  2,  3,
                             4,  4,  1,  2,
                             1,  1,  3,  4,  1])
    )).float().cuda()

In [97]:
from src.p_data_augmentation import PowerPIL
from src.p2_dataload import KaggleAmazonDataset
from src.p_model_selection import train_valid_split
from src.p_sampler import SubsetSampler, balance_weights

In [98]:
# Setting random seeds for reproducibility. (Caveat, some CuDNN algorithms are non-deterministic)
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)
np.random.seed(1337)
random.seed(1337)

##############################################################
## Loading the dataset

## Augmentation + Normalization for full training
ds_transform_augmented = transforms.Compose([
                 transforms.RandomSizedCrop(224),
                 PowerPIL(),
                 transforms.ToTensor(),
                 normalize
])

## Normalization only for validation and test
ds_transform_raw = transforms.Compose([
                 transforms.Scale(224),
                 transforms.ToTensor(),
                 normalize
                 ])

####     #########     ########     ###########     #####

X_train = KaggleAmazonDataset('./data/train.csv','./data/train-jpg/','.jpg',
                             ds_transform_augmented
                             )
X_val = KaggleAmazonDataset('./data/train.csv','./data/train-jpg/','.jpg',
                             ds_transform_raw
                             )

# Creating a validation split
train_idx, valid_idx = train_valid_split(X_train, 0.2)

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetSampler(valid_idx)

######    ##########    ##########    ########    #########

# Both dataloader loads from the same dataset but with different indices
train_loader = DataLoader(X_train,
                      batch_size=batch_size,
                      sampler=train_sampler,
                      num_workers=4,
                      pin_memory=True)

valid_loader = DataLoader(X_val,
                      batch_size=batch_size,
                      sampler=valid_sampler,
                      num_workers=4,
                      pin_memory=True)

In [99]:
def train(epoch, train_loader, encoder, decoder, criterion, optimizer):
    encoder.eval()
    decoder.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target, requires_grad=False)
        optimizer.zero_grad()
        encoded = encoder(data)
        output = decoder(encoded)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [100]:
for epoch in range(1, 10):
    train(epoch, train_loader, encoderCNN, decoderRNN, criterion, optimizer)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.435556
Train Epoch: 1 [640/40479 (2%)]	Loss: 0.409556
Train Epoch: 1 [1280/40479 (4%)]	Loss: 0.369687
Train Epoch: 1 [1920/40479 (6%)]	Loss: 0.339583
Train Epoch: 1 [2560/40479 (8%)]	Loss: 0.303115
Train Epoch: 1 [3200/40479 (10%)]	Loss: 0.287817
Train Epoch: 1 [3840/40479 (12%)]	Loss: 0.251417
Train Epoch: 1 [4480/40479 (14%)]	Loss: 0.244486
Train Epoch: 1 [5120/40479 (16%)]	Loss: 0.251688
Train Epoch: 1 [5760/40479 (18%)]	Loss: 0.218834
Train Epoch: 1 [6400/40479 (20%)]	Loss: 0.228922
Train Epoch: 1 [7040/40479 (22%)]	Loss: 0.257038
Train Epoch: 1 [7680/40479 (24%)]	Loss: 0.240509
Train Epoch: 1 [8320/40479 (26%)]	Loss: 0.249612
Train Epoch: 1 [8960/40479 (28%)]	Loss: 0.212539
Train Epoch: 1 [9600/40479 (30%)]	Loss: 0.213244
Train Epoch: 1 [10240/40479 (32%)]	Loss: 0.232863
Train Epoch: 1 [10880/40479 (34%)]	Loss: 0.229000
Train Epoch: 1 [11520/40479 (36%)]	Loss: 0.217483
Train Epoch: 1 [12160/40479 (38%)]	Loss: 0.240352
Train Epoch: 1 [12800/404

Train Epoch: 4 [8320/40479 (26%)]	Loss: 0.192985
Train Epoch: 4 [8960/40479 (28%)]	Loss: 0.232153
Train Epoch: 4 [9600/40479 (30%)]	Loss: 0.209456
Train Epoch: 4 [10240/40479 (32%)]	Loss: 0.212727
Train Epoch: 4 [10880/40479 (34%)]	Loss: 0.218378
Train Epoch: 4 [11520/40479 (36%)]	Loss: 0.216620
Train Epoch: 4 [12160/40479 (38%)]	Loss: 0.208027
Train Epoch: 4 [12800/40479 (40%)]	Loss: 0.234140
Train Epoch: 4 [13440/40479 (42%)]	Loss: 0.216869
Train Epoch: 4 [14080/40479 (43%)]	Loss: 0.192126
Train Epoch: 4 [14720/40479 (45%)]	Loss: 0.213921
Train Epoch: 4 [15360/40479 (47%)]	Loss: 0.208655
Train Epoch: 4 [16000/40479 (49%)]	Loss: 0.221401
Train Epoch: 4 [16640/40479 (51%)]	Loss: 0.226007
Train Epoch: 4 [17280/40479 (53%)]	Loss: 0.245658
Train Epoch: 4 [17920/40479 (55%)]	Loss: 0.216689
Train Epoch: 4 [18560/40479 (57%)]	Loss: 0.204786
Train Epoch: 4 [19200/40479 (59%)]	Loss: 0.231551
Train Epoch: 4 [19840/40479 (61%)]	Loss: 0.233882
Train Epoch: 4 [20480/40479 (63%)]	Loss: 0.237672
Tra

Train Epoch: 7 [16640/40479 (51%)]	Loss: 0.211867
Train Epoch: 7 [17280/40479 (53%)]	Loss: 0.213061
Train Epoch: 7 [17920/40479 (55%)]	Loss: 0.182352
Train Epoch: 7 [18560/40479 (57%)]	Loss: 0.204680
Train Epoch: 7 [19200/40479 (59%)]	Loss: 0.204601
Train Epoch: 7 [19840/40479 (61%)]	Loss: 0.204036
Train Epoch: 7 [20480/40479 (63%)]	Loss: 0.214126
Train Epoch: 7 [21120/40479 (65%)]	Loss: 0.235875
Train Epoch: 7 [21760/40479 (67%)]	Loss: 0.211310
Train Epoch: 7 [22400/40479 (69%)]	Loss: 0.219646
Train Epoch: 7 [23040/40479 (71%)]	Loss: 0.223003
Train Epoch: 7 [23680/40479 (73%)]	Loss: 0.220125
Train Epoch: 7 [24320/40479 (75%)]	Loss: 0.221888
Train Epoch: 7 [24960/40479 (77%)]	Loss: 0.193348
Train Epoch: 7 [25600/40479 (79%)]	Loss: 0.231392
Train Epoch: 7 [26240/40479 (81%)]	Loss: 0.215880
Train Epoch: 7 [26880/40479 (83%)]	Loss: 0.220085
Train Epoch: 7 [27520/40479 (85%)]	Loss: 0.259755
Train Epoch: 7 [28160/40479 (87%)]	Loss: 0.229210
Train Epoch: 7 [28800/40479 (89%)]	Loss: 0.228965
